In [2]:
import bs4
import lxml
import pandas
import urllib
import pandas as pd
import re

from urllib import request

ModuleNotFoundError: No module named 'lxml'

### Automatize the creation of separate tables as dataframe

In [1]:
# URL of the Wikipedia page containing the tables
ranking = "https://fr.wikipedia.org/wiki/Classement_mondial_ITTF"

# Fetch and parse the web page
request_text = request.urlopen(ranking).read()
soup = bs4.BeautifulSoup(request_text, "lxml")

# Find all tables on the page
tables = soup.find_all("table")

# Loop to process and export each table
for idx, table in enumerate(tables, start=1):
    # Identify the title of the table (if available)
    caption = table.find("caption").get_text(strip=True) if table.find("caption") else f"Table_{idx}"
    print(f"Processing Table {idx}, Title: {caption}")
    
    # Clean the caption: remove the part [number] and keep the relevant part before it
    cleaned_caption = re.sub(r"\[.*?\]", "", caption).strip()  # Remove [number] and any extra spaces

    # Extract data from the table
    rows = table.find_all("tr")
    data = []
    for row in rows:
        # Collect all cells (both th and td for headers and data)
        cells = row.find_all(["th", "td"])
        cell_data = [cell.get_text(strip=True) for cell in cells]
        if cell_data:
            data.append(cell_data)

    if not data:
        print(f"Table {idx} is empty or improperly structured. Skipping.")
        continue

    # Dynamically generate headers
    headers = data[0]  # Use the first row as headers
    data = data[1:]  # Remaining rows are data

    # Ensure headers and data alignment
    max_columns = max(len(row) for row in data)
    if len(headers) < max_columns:
        headers += [f"Colonne {i+1}" for i in range(len(headers), max_columns)]  # Pad missing headers
    elif len(headers) > max_columns:
        headers = headers[:max_columns]  # Truncate excess headers

    # Create DataFrame and clean out header duplication if necessary
    try:
        df = pd.DataFrame(data, columns=headers)  # Exclude header row if present in data
    except ValueError as e:
        print(f"Error creating DataFrame for Table {idx}: {e}")
        continue  # Skip problematic tables

    # Display the first 5 rows as a preview
    print(df.head(5))

    # Export DataFrame to a CSV file
    output_file = f"{cleaned_caption.replace(' ', '_')}.csv"
    df.to_csv(output_file, index=False, encoding="utf-8")
    print(f"Table {idx} exported to {output_file}\n")

print("All tables processed and exported.")


NameError: name 'request' is not defined